In [413]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import Sequential
from time import time

In [601]:
# Import dataset
train = pd.read_csv('optdigits.tra', header=None)
test = pd.read_csv('optdigits.tes', header=None)

In [602]:
# Sort x and y
train_x = train.iloc[:, 0:64]
train_y = train[64]
test_x = test.iloc[:, 0:64]
test_y = test[64]

In [603]:
# Standardization
train_x = train_x.astype('float32') / 16.
test_x = test_x.astype('float32') / 16.

In [604]:
# Split training set into training set and validation set
new_train_x, val_x, new_train_y, val_y = train_test_split(train_x, train_y, test_size=0.2, random_state=1)

In [605]:
# Changed dataset into numpy
new_train_x = new_train_x.to_numpy()
new_train_y = new_train_y.to_numpy()
val_x = val_x.to_numpy()
val_y = val_y.to_numpy()
test_x = test_x.to_numpy()
test_y = test_y.to_numpy()

In [606]:
# Change the shape of y data set
new_train_y = to_categorical(new_train_y)
test_y = to_categorical(test_y)
val_y = to_categorical(val_y)

In [607]:
# Creat a model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(30, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])

In [608]:
# Set SGD as the optimizer
opt = SGD(learning_rate=0.3, momentum=0.3, nesterov=False)

In [609]:
# Early Stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                              min_delta=0, patience=0, 
                              verbose=0, mode='auto')

In [610]:
# Model Compiling
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['accuracy'])

In [611]:
# Fit Model
start = time()
model.fit(new_train_x, new_train_y, epochs=180, batch_size=15,
          callbacks = [early_stop], validation_data=(val_x, val_y))
print(time()-start)

Train on 3058 samples, validate on 765 samples
Epoch 1/180
3058/3058 [==============================] - 1s 283us/sample - loss: 0.0844 - accuracy: 0.3450 - val_loss: 0.0771 - val_accuracy: 0.5373
Epoch 2/180
3058/3058 [==============================] - 0s 151us/sample - loss: 0.0661 - accuracy: 0.6024 - val_loss: 0.0551 - val_accuracy: 0.6706
Epoch 3/180
3058/3058 [==============================] - 0s 149us/sample - loss: 0.0452 - accuracy: 0.7665 - val_loss: 0.0354 - val_accuracy: 0.8706
Epoch 4/180
3058/3058 [==============================] - 0s 152us/sample - loss: 0.0279 - accuracy: 0.9065 - val_loss: 0.0215 - val_accuracy: 0.9307
Epoch 5/180
3058/3058 [==============================] - 0s 141us/sample - loss: 0.0185 - accuracy: 0.9287 - val_loss: 0.0150 - val_accuracy: 0.9412
Epoch 6/180
3058/3058 [==============================] - 0s 154us/sample - loss: 0.0143 - accuracy: 0.9359 - val_loss: 0.0126 - val_accuracy: 0.9542
Epoch 7/180
3058/3058 [==============================] - 0s

In [613]:
# Distribution of training set  >>> counted with for loops
dist_train = [376,389,380,389,387,376,377,387,380,382]
# Distribution of testing set
dist_test = [178,182,177,183,181,182,181,179,174,180]

In [632]:
# Accuracay of overall classification accuracy and class accuracy
def accuracy_overall(pred, actual, num_classes):
    result = int()
# Overall classfication accuracy
    overall = np.where(pred == actual)[0]
    overall_acc = float(len(overall))/len(pred)
    result = overall_acc
    return result

def accuracy(pred, actual, num_classes):
# Class accuracy
    result = []
    classes = np.array([float(0)]*10)
    for i in range(0,len(pred)):
        id_classes = int(actual[i])
        if pred[i] == actual[i]:
            classes[id_classes] = classes[id_classes] + 1
    classes_acc = classes/num_classes
    result.append(classes_acc)
    return result

In [633]:
# confusion matrix
def confusion_matrix(pred, actual):
    matrix = np.array([int(0)]*10*10).reshape(10,10)
    for i in range(0,10):
        pred_i = pred[np.where(np.array(actual) == i)]
        conf = []
        for j in range(0,10):
            length = len(np.where(pred_i == j)[0])
            conf.append(length)
        matrix[i,] = conf
    return matrix

In [634]:
test_y = test[64]
test_y = test_y.to_numpy()

In [635]:
predict_train = model.predict(train_x)
predict_train_class = predict_train.argmax(axis=-1)
acc_train_overall = accuracy_overall(predict_train_class, train_y, dist_train)
acc_train_class = accuracy(predict_train_class, train_y, dist_train)
conf_train = confusion_matrix(predict_train_class, train_y)


predict_test = model.predict(test_x)
predict_test_class = predict_test.argmax(axis=-1)
acc_test_overall = accuracy_overall(predict_test_class, test_y, dist_test)
acc_test_class = accuracy(predict_test_class, test_y, dist_test)
conf_test = confusion_matrix(predict_test_class, test_y)

In [639]:
#output results
print("training set overall accuracy" + "\n")
print(acc_train_overall)
print("\n")
print("training set class accuracy" + "\n")
print(acc_train_class)
print("\n")
print("training set confusion matrix" + "\n")
print(conf_train)
print("\n")
print("testing set overall accuracy" + "\n")
print(acc_test_overall)
print("\n")
print("testing set class accuracy" + "\n")
print(acc_test_class)
print("\n")
print("testing set confusion matrix" + "\n")
print(conf_test)

training set overall accuracy

0.9707036358880461


training set class accuracy

[array([0.99202128, 0.95372751, 0.98421053, 0.96915167, 0.9379845 ,
       0.98138298, 0.9867374 , 0.98966408, 0.96842105, 0.94502618])]


training set confusion matrix

[[373   0   0   0   1   0   1   0   1   0]
 [  0 371   3   0   1   1   0   3   6   4]
 [  0   0 374   0   0   0   1   2   2   1]
 [  0   0   1 377   0   6   0   0   1   4]
 [  1   2   0   0 363   0   4   0   6  11]
 [  0   1   2   1   0 369   0   0   0   3]
 [  1   3   0   0   1   0 372   0   0   0]
 [  0   0   1   1   1   0   0 383   0   1]
 [  0   4   0   2   3   2   1   0 368   0]
 [  1   4   1   3   4   1   0   3   4 361]]


testing set overall accuracy

0.9465776293823038


testing set class accuracy

[array([0.97191011, 0.93956044, 0.97175141, 0.91256831, 0.97790055,
       0.98351648, 0.96132597, 0.92178771, 0.8908046 , 0.93333333])]


testing set confusion matrix

[[173   0   0   0   1   4   0   0   0   0]
 [  0 171   2   0   0   1